### Passo 1 - Importar Arquivos e Bibliotecas

In [ ]:
import pandas as pd
import os
import smtplib
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText
import numpy as np

# 1. Emails
emails = pd.read_excel('Bases de Dados/Emails.xlsx')
emails['E-mail'] = emails['E-mail'].str.replace('pythonimpressionador', 'carlos.seiya16') 
display(emails)

# 2. Lojas
lojas = pd.read_csv('Bases de Dados/Lojas.csv', encoding=r'latin-1', sep=';')
lojas_mesclado = lojas.merge(emails, on='Loja')
display(lojas_mesclado)

# 3. Vendas
vendas = pd.read_excel('Bases de Dados/Vendas.xlsx')
vendas_mesclado = vendas.merge(lojas, on='ID Loja')
display(vendas_mesclado)
display(vendas_mesclado.info())

### Passo 2 - Criar função que envia o e-mail personalizado, de acordo com a hierarquia do destinatário (que pode ser a Gerência ou a Diretoria)

In [ ]:
def enviar_email(hierarquia):
    
    def incluir_anexo(msg, caminho_arquivo, hierarquia):
        if hierarquia == 'diretoria':
            nome_arquivo = caminho_arquivo.split(f'/')[1]
        elif hierarquia == 'gerencia':
            nome_arquivo = caminho_arquivo.split(f'/')[2]
        attachment = open(caminho_arquivo, "rb")
        p = MIMEBase('application', 'octet-stream')
        p.set_payload((attachment).read())
        encoders.encode_base64(p)
        p.add_header('Content-Disposition', "attachment; filename= %s" % nome_arquivo)
        msg.attach(p)
        
    msg = MIMEMultipart()
    msg['From'] = 'carlos.seiya16@gmail.com'
    
    if hierarquia == 'diretoria':
        mensagem = f'''
<p>Prezados(as),</p>

<p>Segue em anexo o ranking das lojas do dia {dia_indicador.day}/{dia_indicador.month}.</p>

<p>A melhor loja do dia foi: {melhor_loja_dia} com R${valor_melhor_loja_dia:.2f} de faturamento.</p>
<p>A pior loja do dia foi: {pior_loja_dia} com R${valor_pior_loja_dia:.2f} de faturamento.</p>

<p>No ano, a melhor loja é {melhor_loja} com R${valor_melhor_loja:.2f} de faturamento acumulado e a pior é {pior_loja} com R${valor_pior_loja:.2f} de faturamento acumulado.</p>

<p>Qualquer dúvida fico à disposição.</p>

<p>Att.,</p>

Seiya
'''
        assunto = f'Ranking Lojas Dia {dia_indicador.day}/{dia_indicador.month}'
        destinatario = 'carlos.seiya16+diretoria@gmail.com'
        # incluindo anexos
        incluir_anexo(msg, fr'Backup Arquivos Lojas/{dia_indicador.month}_{dia_indicador.day}_Faturamento Lojas Dia.xlsx', 'diretoria')
        incluir_anexo(msg, fr'Backup Arquivos Lojas/{dia_indicador.month}_{dia_indicador.day}_Faturamento Lojas Ano.xlsx', 'diretoria')
    
    elif hierarquia == 'gerencia':
        mensagem = f'''
<p>Bom dia, {gerente}</p>

<p>O resultado de <b>ontem (dia {dia_indicador.day}/{dia_indicador.month})</b> da <b>Loja {loja}</b> foi:</p>

<table>
    <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
    </tr>
    <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{produtos_distintos_dia}</td>
        <td style="text-align: center">{meta_produtos_distintos_dia}</td>
        <td style="text-align: center"><font color="{cor_div_dia}">◙</font></td>
    </tr>
    <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticketmedio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_tm_dia}">◙</font></td>
    </tr>
</table>
<br>
<table>
    <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
    </tr>
    <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{produtos_distintos_ano}</td>
        <td style="text-align: center">{meta_produtos_distintos_ano}</td>
        <td style="text-align: center"><font color="{cor_div_ano}">◙</font></td>
    </tr>
    <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticketmedio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_tm_ano}">◙</font></td>
    </tr>
</table>

<p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

<p>Qualquer dúvida, estou à disposição.</p>

<p>Att.,</p>

Fulano
'''
        assunto = f'One Page Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
        destinatario = email_gerente
        # incluindo anexo
        incluir_anexo(msg, fr'Backup Arquivos Lojas/{loja}/{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx', 'gerencia')
        
    # definição de variáveis
    msg['Subject'] = assunto
    msg['To'] = destinatario
    senha = 'squxtqzlsupoctzy'
    
    # configurações do e-mail
    msg.attach(MIMEText(mensagem, 'html'))
    
    try:
        server = smtplib.SMTP("smtp.gmail.com: 587")
        server.starttls()
        server.login(msg['From'], senha)
        server.sendmail(msg['From'], msg['To'], msg.as_string().encode('utf-8'))
        server.close()
        print('Email enviado com sucesso!')
    except:
        print("Falha no envio. Tente novamente.")

### Passo 3 - Código principal
- Criar uma Tabela para cada Loja, e Definir o dia do Indicador
- Salvar a planilha na pasta de backup
- Calcular o indicador para cada uma das lojas
- Enviar por e-mail para o gerente
- Automatizar todas as lojas

In [ ]:
# para cada loja única: se tiver a pasta, só inserir o arquivo do dia_indicador; caso contrário, criar e depois inserir

# 1. dia_indicador manual
# dia_indicador = str(input('Digite o dia que gostaria de analisar (formato: yyyy-mm-dd): '))
# dia_indicador = datetime.strptime(dia_indicador, '%Y-%m-%d')

# 2. dia_indicador automático
dia_indicador = vendas_mesclado['Data'].max() # última dia disponível na tabela de vendas_mesclado
vendas_dia = vendas_mesclado.loc[vendas_mesclado['Data']==dia_indicador, :]

lista_pastas = os.listdir('Backup Arquivos Lojas')
texto_pastas = ''.join(nome for nome in lista_pastas)

# Para cada uma das lojas, fazer tudo e no final enviar o email
# lojas_mesclado = lojas_mesclado.head(2) # enviar para duas primeiras lojas. Tirar essa linha caso queira rodar para todas as lojas!
lojas_mesclado = lojas_mesclado[lojas_mesclado['Loja']=='Norte Shopping'].reset_index(drop=True) # enviar somente para a Norte Shopping. Tirar essa linha caso queira rodar para todas as lojas! 
for i, loja in enumerate(lojas_mesclado['Loja']):
    
    id_loja = lojas_mesclado.loc[i, 'ID Loja']
    
    # 1. Análise do dia
    vendas_loja_dia = vendas_dia.loc[vendas_dia['ID Loja'] == id_loja]
    
    # 1.1. Faturamento
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    meta_faturamento_dia = 1000
    if faturamento_dia >= meta_faturamento_dia:
        print(f'{loja} Superou a meta de Faturamento do dia: ', faturamento_dia)
        cor_fat_dia = 'green'
    elif faturamento_dia < meta_faturamento_dia:
        print(f'{loja} Abaixo da meta de Faturamento do dia: ', faturamento_dia)
        cor_fat_dia = 'red'
        
    # 1.2. Diversidade de produtos
    produtos_distintos_dia = len(vendas_loja_dia['Produto'].unique())
    meta_produtos_distintos_dia = 4
    if produtos_distintos_dia >= meta_produtos_distintos_dia:
        print(f'{loja} Superou a meta de Diversidade de Produtos do dia: ', produtos_distintos_dia)
        cor_div_dia = 'green'
    elif produtos_distintos_dia < meta_produtos_distintos_dia:
        print(f'{loja} Abaixo da meta de Diversidade de Produtos do dia: ', produtos_distintos_dia)
        cor_div_dia = 'red'
    
    # 1.3. Ticket médio por venda
    vendas_loja_dia_agrupado = vendas_loja_dia.groupby('Código Venda').sum()
    ticketmedio_dia = vendas_loja_dia_agrupado['Valor Final'].mean()
    if ticketmedio_dia is np.nan:
        ticketmedio_dia = 0
    meta_ticketmedio_dia = 500
    if ticketmedio_dia >= meta_ticketmedio_dia:
        print(f'{loja} Superou a meta de Ticket Médio do dia: ', ticketmedio_dia)
        cor_tm_dia = 'green'
    elif ticketmedio_dia < meta_ticketmedio_dia:
        print(f'{loja} Abaixo da meta de Ticket Médio do dia: ', ticketmedio_dia)
        cor_tm_dia = 'red'
    
    # 2. Análise do ano
    vendas_loja_ano = vendas_mesclado.loc[vendas_mesclado['ID Loja'] == id_loja]
    
    # 2.1. Faturamento
    faturamento_ano = vendas_loja_ano['Valor Final'].sum()
    meta_faturamento_ano = 1650000
    if faturamento_ano >= meta_faturamento_ano:
        print(f'{loja} Superou a meta de Faturamento do ano: ', faturamento_ano)
        cor_fat_ano = 'green'
    else:
        print(f'{loja} Abaixo da meta de Faturamento do ano: ', faturamento_ano)
        cor_fat_ano = 'red'
        
    # 2.2. Diversidade de produtos
    produtos_distintos_ano = len(vendas_loja_ano['Produto'].unique())
    meta_produtos_distintos_ano = 120
    if produtos_distintos_ano >= meta_produtos_distintos_ano:
        print(f'{loja} Superou a meta de Diversidade de Produtos do ano: ', produtos_distintos_ano)
        cor_div_ano = 'green'
    elif produtos_distintos_ano < meta_produtos_distintos_ano:
        print(f'{loja} Abaixo da meta de Diversidade de Produtos do ano: ', produtos_distintos_ano)
        cor_div_ano = 'red'
    
    # 2.3. Ticket médio por venda
    vendas_loja_ano_agrupado = vendas_loja_ano.groupby('Código Venda').sum()
    ticketmedio_ano = vendas_loja_ano_agrupado['Valor Final'].mean()
    if ticketmedio_ano is np.nan:
        ticketmedio_ano = 0
    meta_ticketmedio_ano = 500
    if ticketmedio_ano >= meta_ticketmedio_ano:
        print(f'{loja} Superou a meta de Ticket Médio do ano: ', ticketmedio_ano)
        cor_tm_ano = 'green'
    elif ticketmedio_ano < meta_ticketmedio_ano:
        print(f'{loja} Abaixo da meta de Ticket Médio do ano: ', ticketmedio_ano)
        cor_tm_ano = 'red'
    
    # condicional: se a loja ainda não tiver uma pasta, criar antes
    if loja not in texto_pastas:
        # criar a pasta antes
        os.mkdir(f'Backup Arquivos Lojas/{loja}')
    # vendas_loja_dia.to_excel(f'Backup Arquivos Lojas/{loja}/{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx', index=False)
    vendas_loja_ano.to_excel(f'Backup Arquivos Lojas/{loja}/{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx', index=False)
    
    # enviar o email
    gerente = lojas_mesclado.loc[i, 'Gerente']
    email_gerente = lojas_mesclado.loc[i, 'E-mail']
    enviar_email('gerencia')

### Passo 4 - Criar ranking para Diretoria

In [ ]:
# Definição das variáveis
# 7.1. Considerando somente o dia: 
vendas_dia_agrupada = vendas_dia.groupby(['Loja'], as_index=False).agg({'Valor Final': 'sum'}).sort_values('Valor Final', ascending=False, ignore_index=True)
vendas_dia_agrupada = vendas_dia_agrupada.rename(columns={'Valor Final': 'Faturamento'})
melhor_loja_dia = vendas_dia_agrupada.loc[0, 'Loja']
valor_melhor_loja_dia = vendas_dia_agrupada.loc[0, 'Faturamento']
pior_loja_dia = vendas_dia_agrupada.loc[len(vendas_dia_agrupada)-1, 'Loja']
valor_pior_loja_dia = vendas_dia_agrupada.loc[len(vendas_dia_agrupada)-1, 'Faturamento']

display(vendas_dia_agrupada)

vendas_dia_agrupada.to_excel(f'Backup Arquivos Lojas/{dia_indicador.month}_{dia_indicador.day}_Faturamento Lojas Dia.xlsx', index=False)

# 7.2. Considerando o ano todo
vendas_agrupada = vendas_mesclado.groupby(['Loja'], as_index=False).agg({'Valor Final': 'sum'}).sort_values('Valor Final', ascending=False, ignore_index=True)
vendas_agrupada = vendas_agrupada.rename(columns={'Valor Final': 'Faturamento'})
melhor_loja = vendas_agrupada.loc[0, 'Loja']
valor_melhor_loja = vendas_agrupada.loc[0, 'Faturamento']
pior_loja = vendas_agrupada.loc[len(vendas_agrupada)-1, 'Loja']
valor_pior_loja = vendas_agrupada.loc[len(vendas_agrupada)-1, 'Faturamento']

display(vendas_agrupada)

vendas_agrupada.to_excel(f'Backup Arquivos Lojas/{dia_indicador.month}_{dia_indicador.day}_Faturamento Lojas Ano.xlsx', index=False)

### Passo 5 - Enviar e-mail para diretoria

In [ ]:
enviar_email('diretoria')